### Create Object Points

Write a method that will precreate object points for 9 x 6 chessboard.

In [8]:
def draw_corners(image, corners, ret, fname, ny=9, nx=6):
    '''
    Draw corners of image
    '''
    cv2.drawChessboardCorners(image, (nx,ny), corners, ret)
    plt.imshow(image)
    plt.show()
    # cv2.waitKey(500)
    print('Found corners for %s' % fname)

def save_image(image, idx):
    write_name = 'output_images/corners_found'+str(idx)+'.jpg'
    cv2.imwrite(write_name, image)
    
def get_object_points():
    nx, ny = 6, 9
    
    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    
    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    
    # find corners on all test images provided to calibrate camera
    for idx, fname in enumerate(images):
        image = cv2.imread(fname)
        
        # convert all images into gray
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # find chess board corners 
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
        
        # save chess board corners
        if ret == True:
            # save object points and it's corners
            objpoints.append(objp)
            imgpoints.append(corners)
            
            draw_corners(image, corners, ret, fname, 9, 6)
            save_image(image, idx)
        else:
            print('Warning: ret = %s for %s' % (ret, fname))
    cv2.destroyAllWindows()        
            
    return objpoints, imgpoints

def calibrate_camera():
    # Get objpoints, imgpoints
    objpoints, imgpoints = get_object_points()
    
    # Test undistortion on an image
    img = cv2.imread('camera_cal/test_image.jpg')
    img_size = (img.shape[1], img.shape[0])
    
    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    save_mtx_dist(mtx, dist)
    
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite('output_images/test_undist.jpg',dst)

    draw_two_images(img, dst)

    return mtx, dist

def draw_two_images(img, dst):
    #dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(dst)
    ax2.set_title('Undistorted Image', fontsize=30)
    plt.show()

def save_mtx_dist(mtx, dist):
    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    dist_pickle = {}
    dist_pickle["mtx"], dist_pickle["dist"] = mtx, dist
    pickle.dump( dist_pickle, open( "output_images/calibrate_camera.p", "wb" ) )
    
    print("Saved calibrate_camera file")

In [9]:
import numpy as np
import cv2
import glob
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import pickle
print ("Imported all libraries!")

print("Calibrating the camera...")
calibrate_camera()

Imported all libraries!
Calibrating the camera...
Found corners for camera_cal/calibration0.jpg
Found corners for camera_cal/calibration10.jpg
Found corners for camera_cal/calibration11.jpg
Found corners for camera_cal/calibration12.jpg
Found corners for camera_cal/calibration13.jpg
Found corners for camera_cal/calibration14.jpg
Found corners for camera_cal/calibration15.jpg
Found corners for camera_cal/calibration16.jpg
Found corners for camera_cal/calibration17.jpg
Found corners for camera_cal/calibration18.jpg
Found corners for camera_cal/calibration19.jpg
Found corners for camera_cal/calibration2.jpg
Found corners for camera_cal/calibration3.jpg
Found corners for camera_cal/calibration6.jpg
Found corners for camera_cal/calibration7.jpg
Found corners for camera_cal/calibration8.jpg
Found corners for camera_cal/calibration9.jpg
Saved calibrate_camera file


(array([[  1.15400811e+03,   0.00000000e+00,   6.69729509e+02],
        [  0.00000000e+00,   1.14807424e+03,   3.85688295e+02],
        [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]]),
 array([[ -2.41271966e-01,  -5.18033940e-02,  -1.15812812e-03,
          -1.27935984e-04,   2.47030758e-02]]))